In [109]:
import re
import pandas as pd

In [110]:
# Self-Login Scraping that didn't work

'''
from dotenv import load_dotenv
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

load_dotenv()

username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

# service = Service(executable_path="chromedriver.exe")
driver = webdriver.Safari()

driver.get("https://intranet.cs.hku.hk/csintranet/login.jsp")

driver.find_element(By.NAME, "j_username").send_keys(username)
driver.find_element(By.NAME, "j_password").send_keys(password)
driver.find_element(By.NAME, "j_password").send_keys(Keys.RETURN)


time.sleep(5)

driver.quit()

# <input type="text" name="j_username" size="25" maxlength="8" class="input">
'''

'\nfrom dotenv import load_dotenv\nimport os\nimport time\nfrom selenium import webdriver\nfrom selenium.webdriver.common.by import By\nfrom selenium.webdriver.common.keys import Keys\nimport pandas as pd\n\nload_dotenv()\n\nusername = os.getenv(\'USERNAME\')\npassword = os.getenv(\'PASSWORD\')\n\n# service = Service(executable_path="chromedriver.exe")\ndriver = webdriver.Safari()\n\ndriver.get("https://intranet.cs.hku.hk/csintranet/login.jsp")\n\ndriver.find_element(By.NAME, "j_username").send_keys(username)\ndriver.find_element(By.NAME, "j_password").send_keys(password)\ndriver.find_element(By.NAME, "j_password").send_keys(Keys.RETURN)\n\n\ntime.sleep(5)\n\ndriver.quit()\n\n# <input type="text" name="j_username" size="25" maxlength="8" class="input">\n'

In [111]:
# setup df
df = pd.DataFrame(columns=[
    'University',
    'University-Country',
    'Exchange-Course-Code',
    'Exchange-Course-Title',
    'HKU-Course-Code',
    'HKU-Course-Title'
])

In [112]:
#regex checking and functions for those particular checks
def particular_cpsc_cleaning(text):
    text = re.sub(r'CompSci', 'COMPSCI', text)
    text = re.sub(r'Comp Sci', 'COMPSCI', text)
    text = re.sub(r'compsci', 'COMPSCI', text)
    text = re.sub(r'COMP_SCI ', 'COMP_SCI', text)
    text = re.sub(r'COMP SCI', 'COMPSCI', text)
    text = re.sub(r'COMPSCI ', 'COMPSCI', text)
    text = re.sub(r'CMPSC ', 'CMPSC', text)
    text = re.sub(r'LE/EECS ', 'LE/EECS', text)
    text = re.sub(r'PSYCH ', 'PSYCH', text)
    text = re.sub(r'ELENG ', 'ELENG', text)
    return text

def after_two_chars(text):
    return text[:2] + text[3:]  

def after_three_chars(text):
    return text[:3] + text[4:]  

def after_four_chars(text):
    return text[:4] + text[5:]  

In [113]:
with open('hkucs-scraped.txt', 'r') as file:
    lines = file.readlines()
    length = len(lines)

i = 0
uni_name = ''
while i < length:
    if lines[i] == 'Foreign University: \n':
        uni_name = lines[i + 1].strip()
        i += 4
    else:
        #
        vals = lines[i].split('\t')
        # cleaning for course code efficiency
        course_info  = particular_cpsc_cleaning(vals[2])
        exchange_course_title = ''

        #cleaning if certain info is missing
        if course_info[2] == ' ': course_info = after_two_chars(course_info)
        elif course_info[3] == ' ': course_info = after_three_chars(course_info)

        elif course_info[4] == ' ' and course_info[:4] != "ST76": course_info = after_four_chars(course_info)

        exchange_course_code = course_info.split(' ')[0]
        if " " in course_info: exchange_course_title = course_info[course_info.index(" ") + 1:]
        new_row = {
                'University': uni_name,
                'University-Country': '',
                'Exchange-Course-Code': exchange_course_code,
                'Exchange-Course-Title': exchange_course_title,
                'HKU-Course-Code': vals[1].strip(),
                'HKU-Course-Title': ''
            }
        df.loc[len(df)] = new_row

        i += 1

In [114]:
# backtracked analysis then cleaning
df.loc[df['University'] == 'Technische Universität München','University'] = 'Technische Universitat Munchen'
df.loc[df['University'] == 'Technische Universität Darmstadt','University'] = 'Technische Universitat Darmstadt'
df.loc[df['University'] == 'The University California, San Diego','University'] = 'University of California, San Diego'
df.loc[df['University'] == 'The University of British Columbia','University'] = 'University of British Columbia'
df.loc[df['University'] == 'The University of Melbourne','University'] = 'University of Melbourne'
df.loc[df['University'] == 'The University of Wiscosin, Madison', 'University'] = 'University of Wisconsin-Madison'
df.loc[df['University'] == 'UC Santa Barbara', 'University'] = 'University of California, Santa Barbara'
df.loc[df['University'] == 'Yonsei university', 'University'] = 'Yonsei University'
df.loc[df['University'] == 'UC Davis', 'University'] = 'University of California, Davis'
df.loc[df['University'] == 'UC Santa Cruz', 'University'] = 'University of California, Santa Cruz'
df.loc[df['University'] == 'Universit√© Paris-Dauphine', 'University'] = 'University of Paris-Dauphine'

In [115]:
# convert to csv
df.to_csv("compsci-credit_transfer_database.csv", index=False)
df.head()

,University,University-Country,Exchange-Course-Code,Exchange-Course-Title,HKU-Course-Code,HKU-Course-Title
0,"Aalto University, School of Science",,CS-C3140,Operating Systems,COMP3230,
1,"Aalto University, School of Science",,CS-E3190,Principles of Algorithmic Techniques D,COMP3251,
2,"Aalto University, School of Science",,CS-C3240,Machine Learning,COMP3314,
3,"Aalto University, School of Science",,CS-C3130,Information Security,COMP3355,
4,"Aalto University, School of Science",,CS-C3170,Web Software Development,CSEL0001,
